# MPC/SBN Database Tests (June 2023)

This code uses the Rubin Observatory replica at University of Washington.

## Getting this notebook

This notebook is designed to be run from the LINCC JupyterHub at:

    https://lsst.dirac.dev

(though you can run it from your own machine with small changes).

The notebook is available from https://github.com/mjuric/mpc-sbn-tests. To obtain it, open a terminal and clone it into your home directory to run it:

    git clone https://github.com/mjuric/mpc-sbn-tests
.

## Connecting and inspecting available tables

In [8]:
import psycopg2 as pg
import pandas as pd
import numpy as np

In [12]:
pd.set_option('display.max_columns', None)

In [3]:
pwd = open("/home/shared/sssc-db-pass.txt").read()
con = pg.connect(database="mpc_sbn", user="sssc", password=pwd, host="epyc.astro.washington.edu", port="5432")

See which tables are available:

In [6]:
tables = pd.read_sql("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'", con)
tables = tables.set_index("table_name")
tables

""
table_name
neocp_els
neocp_events
neocp_obs
neocp_obs_archive
neocp_prev_des
neocp_var
obs_sbn
mpc_orbits
current_identifications


See how many rows each table has (this takes awhile to run -- ~10 minutes or so):

In [9]:
%%time
tables["nrows"] = np.zeros(len(tables), dtype=int)
for table in tables.index:
    df = pd.read_sql(f"SELECT COUNT(*) FROM {table}", con, params=dict(table=table))
    tables["nrows"].loc[table] = df["count"].iloc[0]

In [10]:
tables

,nrows
table_name,
neocp_els,21
neocp_events,182534
neocp_obs,281
neocp_obs_archive,476542
neocp_prev_des,54088
neocp_var,43476
obs_sbn,396010031
mpc_orbits,1284248
current_identifications,1821172


List indices we have on the database:

In [14]:
query = """
SELECT
    tablename,
    indexname,
    indexdef
FROM
    pg_indexes
WHERE
    schemaname = 'public'
ORDER BY
    tablename,
    indexname;
"""
pd.read_sql(query, con)

,tablename,indexname,indexdef
0,current_identifications,current_identifications_packed_primary_provisi...,CREATE INDEX current_identifications_packed_pr...
1,current_identifications,current_identifications_packed_secondary_provi...,CREATE UNIQUE INDEX current_identifications_pa...
2,current_identifications,current_identifications_pkey,CREATE UNIQUE INDEX current_identifications_pk...
3,mpc_orbits,mpc_orbits_pkey,CREATE UNIQUE INDEX mpc_orbits_pkey ON public....
4,mpc_orbits,packed_primary_provisional_idx,CREATE UNIQUE INDEX packed_primary_provisional...
5,mpc_orbits,unpacked_primary_provisional_idx,CREATE UNIQUE INDEX unpacked_primary_provision...
6,neocp_els,neocp_els_created_at_key,CREATE INDEX neocp_els_created_at_key ON publi...
7,neocp_els,neocp_els_desig_key,CREATE UNIQUE INDEX neocp_els_desig_key ON pub...
8,neocp_els,neocp_els_digest2_key,CREATE INDEX neocp_els_digest2_key ON public.n...
9,neocp_els,neocp_els_pkey,CREATE UNIQUE INDEX neocp_els_pkey ON public.n...


Let's get a feel for the available data, by grabbing the top 5 rows of `obs_sbn`:

In [15]:
pd.read_sql("SELECT * FROM obs_sbn LIMIT 5", con)

,id,trksub,trkid,obsid,submission_id,submission_block_id,obs80,status,ref,healpix,permid,provid,artsat,mode,stn,trx,rcv,sys,ctr,pos1,pos2,pos3,poscov11,poscov12,poscov13,poscov22,poscov23,poscov33,prog,obstime,ra,dec,rastar,decstar,obscenter,deltara,deltadec,dist,pa,rmsra,rmsdec,rmsdist,rmspa,rmscorr,delay,rmsdelay,doppler,rmsdoppler,astcat,mag,rmsmag,band,photcat,photap,nucmag,logsnr,seeing,exp,rmsfit,com,frq,disc,subfrm,subfmt,prectime,precra,precdec,unctime,notes,remarks,deprecated,localuse,nstars,prev_desig,prev_ref,rmstime,created_at,updated_at,trkmpc,orbit_id,designation_asterisk,all_pub_ref,shapeocc,obssubid,replacesobsid,group_id
0,47633487,P10aawA,000001oj9Q,Kiv45q0000005jL701000007r,2014-02-27T04:22:18.000_00005jL7,2014-02-27T04:22:18.000_00005jL7_01,43996 C2014 02 26.42238 07 59 25.258+1...,P,MPS 502832,1246427984,43996,None,None,CCD,F51,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2014-02-26 10:08:14,119.855242,18.461572,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2MASS,19.9,None,w,None,None,None,None,None,None,None,None,None,None,None,M92,10,0.001,0.01,None,None,None,None,None,None,None,None,None,2017-07-10 00:00:00+00:00,2022-04-02 21:34:32.859424+00:00,None,None,None,None,None,None,None,None
1,47633488,P10aawA,000001oj9Q,Kiv45q0000005jL701000007s,2014-02-27T04:22:18.000_00005jL7,2014-02-27T04:22:18.000_00005jL7_01,43996 C2014 02 26.43540 07 59 24.901+1...,P,MPS 502832,1246427986,43996,None,None,CCD,F51,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2014-02-26 10:26:59,119.853754,18.461953,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2MASS,19.8,None,w,None,None,None,None,None,None,None,None,None,None,None,M92,10,0.001,0.01,None,None,None,None,None,None,None,None,None,2017-07-10 00:00:00+00:00,2022-04-02 21:34:32.860731+00:00,None,None,None,None,None,None,None,None
2,47633490,VE83847,000001pSO1,Kj9Bii0100005jrb0100004T2,2014-03-10T12:30:56.001_00005jrb,2014-03-10T12:30:56.001_00005jrb_01,43996 C2014 03 10.14990 07 55 59.20 +1...,P,MPS 505761,1247970286,43996,None,None,CCD,G96,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2014-03-10 03:35:51,118.996670,18.754030,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,UCAC4,19.8,None,V,None,None,None,None,None,None,None,None,None,None,None,M92,10,0.010,0.10,None,None,None,None,None,None,None,None,None,2017-07-10 00:00:00+00:00,2022-04-02 21:34:32.864786+00:00,None,None,None,None,None,None,None,None
3,47633491,VE83847,000001pSO1,Kj9Bii0100005jrb0100004T3,2014-03-10T12:30:56.001_00005jrb,2014-03-10T12:30:56.001_00005jrb_01,43996 C2014 03 10.15633 07 55 59.15 +1...,P,MPS 505761,1247971652,43996,None,None,CCD,G96,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2014-03-10 03:45:07,118.996460,18.754140,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,UCAC4,19.7,None,V,None,None,None,None,None,None,None,None,None,None,None,M92,10,0.010,0.10,None,None,None,None,None,None,None,None,None,2017-07-10 00:00:00+00:00,2022-04-02 21:34:32.866534+00:00,None,None,None,None,None,None,None,None
4,47633492,VE83847,000001pSO1,Kj9Bii0100005jrb0100004T4,2014-03-10T12:30:56.001_00005jrb,2014-03-10T12:30:56.001_00005jrb_01,43996 C2014 03 10.16275 07 55 59.06 +1...,P,MPS 505761,1247971652,43996,None,None,CCD,G96,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2014-03-10 03:54:22,118.996080,18.754280,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,UCAC4,19.8,None,V,None,None,None,None,None,None,None,None,None,None,None,M92,10,0.010,0.10,None,None,None,None,None,None,None,None,None,2017-07-10 00:00:00+00:00,2022-04-02 21:34:32.868178+00:00,None,None,None,None,None,None,None,None


Let's give it something more to think about...: